### Boilerplate Imports

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from typing import TypedDict, List, Literal, Optional, Tuple
from langgraph.graph import StateGraph, END
import sys

import sqlite3
from dotenv import load_dotenv
load_dotenv(override=True)

llm = ChatOpenAI(model="gpt-4", temperature=0)

### Module: Database Connection

In [10]:
# Set up SQLite database
conn = sqlite3.connect('customers.db')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT,
    phone TEXT
)
''')
conn.commit()

### Module: Define the State (Shared Memory)

In [11]:
class State(TypedDict):
    user_input: str
    conversation_history: List[Tuple[str, str]]
    bot_response: str
    complete_user_info: Optional[str]
    status: Literal["active","finish"]

### Module: Define Prompt Functions

In [12]:
# Handle User Response
def get_handle_user_response_prompt() -> str:

    # System Prompt
    systemContent = """
    You are a customer service chatbot, who has asked if a customer is a new or existing customer. 

    Customer has responded with: "{customer_response}"

    Your task is to ONLY output one of these exact words: "new", "existing", "pii", or "random"

    Rules:
    1. If customer says they are new -> output only the word "new"
    2. If customer says they are existing -> output only the word "existing"
    3. If customer provides any of the following identifiable information (name, email, phone) -> output only the word "pii"
    4. For any other response or out of scope questions -> output only the word "random"

    IMPORTANT: 
    - Output ONLY ONE WORD from the allowed options: "new", "existing", "random", "pii"
    - Do not include any other text, punctuation, or explanation
    - Do not engage in conversation or answer questions

    """

    # List of Messages
    messages = [("system",systemContent), ("human", "{customer_response}")]

    # Set the Prompt
    prompt = ChatPromptTemplate.from_messages(messages)

    # Return the Prompt
    return prompt


# Ask Customer for more information
def get_more_customer_info_prompt() -> str:

    # System Prompt
    systemContent = """
    You are a friendly and professional customer service chatbot. You have just asked the customer if they are a new or an existing customer.
    The customer has responded that they are either a new or existing customer but hasn't provided all of the identifiable information (name, email, and phone number).

    Your Task:
    - Review the conversation history provided as {conversation_history} to determine which identifiable information (name, email, phone number) the customer has already provided.
    - Politely request the customer to provide only the identifiable information that is missing so that you can assist them further.

    Guidelines:
    - Maintain a polite and courteous tone throughout the interaction.
    - Use clear and concise language to make your request.
    - Ensure the customer feels valued and understood.
    - Encourage the customer to provide the required information without causing frustration.
    - Avoid asking for information the customer has already provided.

    """

    # List of Messages
    messages = [("system", systemContent),  ("human", "{conversation_history}")]

    # Set the Prompt
    prompt = ChatPromptTemplate.from_messages(messages)

    return prompt


# Ask Customer for more information
def get_handle_random_response_prompt() -> str:

    # System Prompt
    systemContent = """
    You are a customer service chatbot, who has asked if a customer is a new or existing customer. 

    If the customer responds with a question or something unrelated:
    - Politely inform them that you're currently unable to assist with that specific request.
    - Example: "I'm sorry, but I can't assist with that at the moment."

    If the customer provides an invalid or unexpected response:
    - Gently let them know that their response wasn't understood.
    - Example: "It seems there was an issue with your response."

    After either case, Review the conversation history {conversation_history} to determine Which identifiable information (name, email, phone number) the customer not provided.
    Politely request the customer to provide only the identifiable information that is missing so that you can assist them further.

    Ensure that your tone remains courteous and helpful throughout the interaction, making the customer feel valued and understood.

    """

    # List of Messages
    messages = [("system", systemContent), ("human", "{conversation_history}")]

    # Create and Return the prompt
    prompt = ChatPromptTemplate.from_messages(messages)

    return prompt


# Greet the Existing Customer
def get_existing_customer_greet_response_prompt() -> str:

    # System Prompt
    systemContent = """
    You are a polite and professional customer service chatbot. 
    You have successfully verified that the customer exists in the database using the information they provided. 
    Your task is to greet the customer warmly and make them feel valued. 
    
    Use a friendly and personalized tone to welcome them back, and express appreciation for their continued engagement with the service. 
    For example, you can say: 
    "Welcome back! We're so glad to see you again. Thank you for being a valued customer."

    Remember - do not ask any question.

    """

    # List of messages
    messages = [("system", systemContent)]

    # Create and return the prompt
    prompt = ChatPromptTemplate.from_messages(messages)
    return prompt


# Extract Customer PII
def get_extract_customer_PII_prompt() -> str:

    # System Prompt
    systemContent = """
    You are a customer service chatbot designed to assist customers by collecting and verifying their identifiable information.

    Inputs:
    - Conversation History: {conversation_history} (a record of the prior exchanges with the customer).
    - Customer Response: "{customer_response}" (the latest message from the customer, which may contain one, two, or all three of the following pieces of information: name, phone number, and email).

    Your Task:
    1. Review the Conversation History:
       - Extract any previously provided identifiable information: Name, Email, Phone Number

    2. Analyze the Customer's Latest Response:
       - Extract any new identifiable information from {customer_response}: Name, Email, Phone Number

    3. Consolidate the Information:
       - Combine the information from both the conversation history and the latest response.
       - If a piece of information was provided earlier and updated in the latest response, use the most recent value.
       - Determine which pieces of information are still missing.
    
    4. Format the Consolidated Information as Follows: name: <Given Name>, email: <Given Email>, phone: <Given Phone Number> 
       - if name is not provided then use: name: ""
       - if email is not provided then use: email: ""
       - if phone is not provided then use: phone: ""
    
    5. If any information is missing after consolidation, leave the corresponding field blank (e.g., name: "").

    Example 1:
    Customer Response: "My name is John Doe, and my email is john.doe@example.com."
    Output:
    name: John Doe, email: john.doe@example.com, phone: ""

    Example 2:
    Customer Response: "Here's my phone number: 0412 345 678."
    Output:
    name: "", email: "", phone: 0412 345 678

    Guideline:
    - Thoroughly analyze both the conversation history and the current response to ensure all available information is extracted.
    - Do not ask the customer for information in this step; simply extract and compile what has been provided.
    - Ensure accuracy by double-checking that the information matches the appropriate categories (e.g., emails are correctly identified as emails).
    - Always ensure the output is clear, concise, and follows the specified format.

    """

    # List of messages
    messages = [("system", systemContent), ("human", "{customer_response} {conversation_history}")]

    # Create and return the prompt
    prompt = ChatPromptTemplate.from_messages(messages)
    return prompt

# Check Customer PII Availability
def get_check_customer_PII_availability_prompt() -> str:

    # System Prompt
    systemContent = """
    You are a customer service chatbot designed to check if a customer has provided all the required identifiable information (name, email, and phone number). 
    The customer has responded with: "{customer_response}", which is formatted as follows:

    Example 1:
    name: John Doe, email: john.doe@example.com, phone: ""

    Example 2:
    name: "", email: "", phone: 0412 345 678

    Your task is to:
    1. Check if the customer has provided all three pieces of information: name, email, and phone number.
    2. If all three are provided, output: all
    3. If any of the three are missing, output: partial

    Examples:
    Input:
    name: John Doe, email: john.doe@example.com, phone: 0412 345 678
    Output: all

    Input:
    name: "", email: john.doe@example.com, phone: 0412 345 678
    Output: partial

    Input:
    name: "", email: "", phone: 0412 345 678
    Output: partial

    Always output only one of the two words: all or partial.

    """

    # List of messages
    messages = [("system", systemContent), ("human", "{customer_response}")]

    # Create and return the prompt
    prompt = ChatPromptTemplate.from_messages(messages)
    return prompt

### Module: Define Nodes

In [13]:
# Entry Node: Handle User Response
def handle_user_response(state: State) -> State:

    # Get the Prompt
    prompt = get_handle_user_response_prompt()

    # Define the Chain
    chain = prompt | llm

    # Invoke the chain
    response = chain.invoke({"customer_response": state["user_input"]})

    # Return the updated dictionary
    return {
        **state,
        "bot_response": response.content,
    }

# Node: Extract Customer PII
def extract_user_pii(state: State) -> State:

    # Get the Prompt
    prompt = get_extract_customer_PII_prompt()

    # Define the Chain
    chain = prompt | llm

    # Invoke the chain
    response = chain.invoke({"customer_response": state["user_input"], "conversation_history":state["conversation_history"]})

    # Return the updated dictionary
    return {
        **state,
        "bot_response": response.content,    # update bot response
        "conversation_history": [*state["conversation_history"], ("assistant", response.content)]   # update conversation history
    }

# Node: Check Customer PII Availability
def check_user_pii_availability(state: State) -> State:

    # Get the Prompt
    prompt = get_check_customer_PII_availability_prompt()

    # Define the Chain
    chain = prompt | llm

    # Invoke the chain
    response = chain.invoke({"customer_response": state["bot_response"]})

    # Store Customer info in case all the PII are provides
    if response.content == "all":
        state["complete_user_info"] = state["bot_response"]

    # Return the updated dictionary
    return {
        **state,
        "bot_response": response.content,    # update bot response
    }

# Node: Greet Existing Customer
def greet_existing_customer(state: State) -> State:

    # Get the Prompt
    prompt = get_existing_customer_greet_response_prompt()
    
    # Define the Chain
    chain = prompt | llm

    # Invoke the chain
    response = chain.invoke({})

    # Return the updated dictionary
    return {
        **state,
        "status": "finish",
        "bot_response": response.content    # update bot response
    }

# Node: Ask Customer for Missing Information
def ask_customer_missing_information(state: State) -> State:

    # Get the Prompt
    prompt = get_more_customer_info_prompt()
    
    # Define the Chain
    chain = prompt | llm

    # Invoke the chain
    response = chain.invoke({"conversation_history": state["conversation_history"]})

    # Return the updated dictionary
    return {
        **state,
        "bot_response": response.content,    # update bot response
        "conversation_history": [*state["conversation_history"], ("assistant",response.content)]
    }

# Node: Ask random response from customer
def handle_customer_random_response(state: State) -> State:

    # Get the Prompt
    prompt = get_handle_random_response_prompt()
    
    # Define the Chain
    chain = prompt | llm

    # Invoke the chain
    response = chain.invoke({"conversation_history": state["conversation_history"]})

    # Return the updated dictionary
    return {
        **state,
        "bot_response": response.content,    # update bot response
        "conversation_history": [*state["conversation_history"], ("assistant",response.content)]
    }


# Node: Add customer to database
def add_customer_to_database(state: State) -> State:
    
    # Parse the input string to extract name, email, and phone
    name = ""
    email = ""
    phone = ""

    # Split the string by commas and process each part
    for part in state["complete_user_info"].split(","):
        part = part.strip()  # Remove leading/trailing whitespace
        if "name:" in part:
            name = part.split(":")[1].strip()
        elif "email:" in part:
            email = part.split(":")[1].strip()
        elif "phone:" in part:
            phone = part.split(":")[1].strip()

    # Connect to the SQLite database
    conn = sqlite3.connect("customers.db")
    cursor = conn.cursor()

    # Insert the customer into the database
    cursor.execute("INSERT INTO customers (name, email, phone) VALUES (?, ?, ?)", (name, email, phone))
    conn.commit()  # Commit the transaction
    conn.close()  # Close the database connection

    # Return the updated dictionary
    return {
        **state,
        "status": "finish",
        "bot_response": "User added successfully!"    # update bot response
    }

### Supporting function to `Query` the Database

In [14]:
def query_database (state: State) -> str:
    
    # Extract the bot_response from the state dictionary
    bot_response = state["bot_response"]

    # Parse the bot_response to extract email and phone
    email = ""
    phone = ""
    
    # Split the bot_response into key-value pairs
    for part in bot_response.split(","):
        part = part.strip()  # Remove leading/trailing whitespace
        if "email:" in part:
            email = part.split(":")[1].strip()
        elif "phone:" in part:
            phone = part.split(":")[1].strip()

    # Check if both email and phone are missing
    if not email and not phone:
        return "no match"

    # Connect to the SQLite database
    conn = sqlite3.connect("customers.db")
    cursor = conn.cursor()

    # Query the database using email and/or phone
    query = "SELECT * FROM customers WHERE "
    conditions = []
    params = []

    if email:
        conditions.append("email = ?")
        params.append(email)
    if phone:
        conditions.append("phone = ?")
        params.append(phone)

    # Combine conditions with OR logic
    query += " OR ".join(conditions)

    # Execute the query
    cursor.execute(query, params)
    result = cursor.fetchone()

    # Close the database connection
    conn.close()

    # Return "match" if a record is found, otherwise "no match"
    return "match" if result else "no match"

### Module: Define Workflow

In [15]:
# Define workflow
workflow = StateGraph(State)

# Add nodes to the workflow
workflow.add_node("node_handle_user_response", handle_user_response)
workflow.add_node("node_extract_user_pii", extract_user_pii)
workflow.add_node("node_check_user_pii_availability", check_user_pii_availability)
workflow.add_node("node_greet_existing_customer", greet_existing_customer)
workflow.add_node("node_ask_customer_missing_information", ask_customer_missing_information)
workflow.add_node("node_handle_customer_random_response", handle_customer_random_response)
workflow.add_node("node_add_customer_to_database", add_customer_to_database)

# Add a conditional edge
workflow.add_conditional_edges(
    "node_handle_user_response",
    lambda state: (
        "node_extract_user_pii" if state["bot_response"] == "pii" else
        "node_handle_customer_random_response" if state["bot_response"] == "random" else
        "node_ask_customer_missing_information" if state["bot_response"] in ["new", "existing"] else
        END  # Default to END if no condition matches (failsafe)
    )
)

# Add a conditional edge
workflow.add_conditional_edges(
    "node_extract_user_pii",
    lambda state: (
        "node_greet_existing_customer" if query_database(state) == "match" else
        "node_check_user_pii_availability"
    )
)

# Add a conditional edge
workflow.add_conditional_edges(
    "node_check_user_pii_availability",
    lambda state: (
        "node_add_customer_to_database" if state["bot_response"] == "all" else
        "node_ask_customer_missing_information" if state["bot_response"] == "partial" else
        END  # Failsafe condition
    )
)

# Add directed edges
workflow.add_edge("node_handle_customer_random_response", END)
workflow.add_edge("node_greet_existing_customer", END)
workflow.add_edge("node_add_customer_to_database", END)
workflow.add_edge("node_ask_customer_missing_information", END)

# Set entry point
workflow.set_entry_point("node_handle_user_response")

# Compile the workflow
app = workflow.compile()

### Model: Main Wrapper

In [16]:
# Initialize Graph State
state = {
    "user_input": "",
    "conversation_history": [("system", "Hi! are you a new or existing customer")],
    "bot_response": "",
    "status":"active"
}

print("AI: Hi! are you a new or existing customer")

# Start Conversation
while state["status"] == "active":

    # Get the user input
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit", "bye", "close"]:
        print("AI: Goodbye!")
        break
    
    else:
        # Add User Input
        state["user_input"] = user_input

        # Update Conversation History
        state["conversation_history"].append(("human", user_input))

        # Invoke the graph
        result = app.invoke(state)

        # Update the state
        state['status'] = result['status']

        # Print the bot response
        sys.stdout.write(f"AI: {result['bot_response']}\n")
        sys.stdout.flush()

AI: Hi! are you a new or existing customer
AI: Hello Arishka, thank you for providing your name. To assist you further, could you please share your email address and phone number? Your information will be handled with utmost confidentiality.
AI: Hello Arishka, thank you for providing your name and email. To assist you further, could you please provide your phone number? This will help us to serve you better.
AI: I'm sorry, but I can't assist with that at the moment. From our conversation, I see that you've provided your name and email address. However, to assist you further, could you please provide your phone number? Your cooperation is greatly appreciated.
AI: Goodbye!
